#### In this problem you, can simulate a simplified airport security system at a busy airport. 
#### Passengers arrive according to a Poisson distribution with λ1 = 5 per minute (i.e., mean interarrival rate µ1 = 0.2 minutes) to the ID/boarding-pass check queue, where there are several servers who each have exponential service time with mean rate µ2 = 0.75 minutes. [Hint: model them as one block that has more than one resource.]  where they go through the personal scanner (time is uniformly distributed between 0.5 minutes and 1 minute).

#### Use the Arena software (PC users) or Python with SimPy (PC or Mac users) to build a simulation of the system, and then vary the number of ID/boarding-pass checkers and personal-check queues to  determine how many are needed to keep average wait times below 15 minutes. [If you’re using SimPy,  or if you have access to a non-student version of Arena, you can use λ1 = 50 to simulate a busier airport.]

In [14]:
import random
import simpy


arrival_lambda_1 = 5
arrival_mu = .2
server_mu = .75
scanner_lower = 0.5
scanner_upper = 1
max_wait = 15
sim_time = 30

num_scanners = 20
num_controllers = 10


In [15]:
class Airport(object):
    """A airport has a limited number of machines (``NUM_MACHINES``) to
    clean cars in parallel.

    Cars have to request one of the machines. When they got one, they
    can start the washing processes and wait for it to finish (which
    takes ``washtime`` minutes).

    """
    def __init__(self, env, num_controllers, num_scanners):
        self.env = env
        self.controller = simpy.Resource(env, num_controllers)
        self.scanner = simpy.Resource(env, num_scanners)
        #self.washtime = washtime


    def controller(self, passenger):
        """The washing processes. It takes a ``car`` processes and tries
        to clean it."""
        yield self.env.timeout(random.expovariate(server_mu))
        
    def scanner(self, passenger):
        yield self.env.timeout(random.uniform(scanner_lower, scanner_upper))

def passenger(env, name, airport):
    """The car process (each car has a ``name``) arrives at the carwash
    (``cw``) and requests a cleaning machine.

    It then starts the washing process, waits for it to finish and
    leaves to never come back ...

    """
    print('%s arrives at the carwash at %.2f.' % (name, env.now))
    with airport.controller.request() as request:
        yield request

        print('%s enters the controller line at %.2f.' % (name, env.now))
        yield env.process(airport.controller(name))

        print('%s leaves the carwash at %.2f.' % (name, env.now))
        
    # what about scanner??
    with airport.scanner.request() as request:
        yield request

        print('%s enters the controller line at %.2f.' % (name, env.now))
        yield env.process(airport.scanner(name))

        print('%s leaves the carwash at %.2f.' % (name, env.now))

        
        
        

def setup(env, num_scanners, num_controllers):

    ## Create the carwash
    #carwash = Carwash(env, num_machines, washtime)

    # create airport
    airport = Airport(env, num_scanners, num_controllers)
    
    # Create 4 initial cars
    #for i in range(4):
    #    env.process(car(env, 'Car %d' % i, carwash))

    
    # Create passengers while the simulation is running
    i = 0 # person 1, not 0 indexed
    
    while True:
        #yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        #i += 1
        #env.process(passenger(env, 'Car %d' % i, airport))
        
        
        arrival_int = random.expovariate(arrival_lambda_1)
        yield env.timeout(arrival_int)
        i += 1
        env.process(passenger(env, 'Passenger %d' % i, airport))

        

In [16]:

# Setup and start the simulation
print('People are arriving...omg!')
random.seed(123)  # This helps reproducing the results

# Create an environment and start the setup process
env = simpy.Environment()
env.process(setup(env, num_controllers, num_scanners))

# Execute!
env.run(until=sim_time)

People are arriving...omg!
Passenger 1 arrives at the carwash at 0.01.
Passenger 1 enters the controller line at 0.01.


TypeError: 'Resource' object is not callable